In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np

# own Modules 
from models_sub_net_ls import LstmMse_LatentSpace
from data_preperator import DataPreperatorPrediction
from data_set import DataSet
from loss_module import LossMseInference
from predictor import PredictorMultiTaskLearning

## Take care of these things before training:
- Select correct path and define droped_features
- Change parameter of model
- Change filed_location

## Parameters

In [2]:
param = {
    "data" : {
        "stake_training_data" : 0.75, 
        "path" : '../../data/phm_data_challenge/01_M01_DC_training.csv',
        "droped_feature": ["stage", "Lot", "runnum", "recipe", "recipe_step",
                            "up time", "ongoing time", 
                            "ETCHSOURCEUSAGE", "ETCHAUXSOURCETIMER", 
                            "ETCHAUX2SOURCETIMER", "FIXTURESHUTTERPOSITION"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "path" : "../../models/MSE_multi_learning/phm_data_InputSize13_LayerLstm2_HiddenLstm100_HiddenFc_pred50_HiddenFc_ls7_Seq150.pt",
        "input_size" : 13,
        "n_hidden_lstm" : 100,
        "sequence_size" : 100,
        "batch_size" : 50,
        "lstm_layer" : 2,
        "n_hidden_fc_pred": 50,
        "n_hidden_fc_ls": 7,
        "dropout_rate_lstm": 0.0,
        "dropout_rate_fc": 0.2
    },
    "anomaly_detection" : {
        "threshold_anomaly" : 0.3,
        "smooth_rate" : 0.05
    },
    "results": {
        "path" : "../visualisation/files/multi_task_phm_old.csv",
    }
}

## Standarize Data
First we have to apply normalisation to data. That is because the model works on the representation given by its input vectors. The scale of those numbers is part of the representation.
We should apply the exact same scaling as for training data. That means storing the scale and offset used with your training data, and using that again. <br>
__The mean and variance for each feature of the training data with which the model was trained (stake: 0.75):__

### Mean and Variance from phm Dataset
droped features="stage", "Lot", "runnum", "recipe", "recipe_step",
                            "up time", "ongoing time", 
                            "ETCHSOURCEUSAGE", "ETCHAUXSOURCETIMER", 
                            "ETCHAUX2SOURCETIMER", "FIXTURESHUTTERPOSITION"

In [55]:
mean_training_data =[ 0.21683119,  0.32121513,  0.31925213,  0.20097501,  0.45164471,  0.22914814,
  0.11604865,  0.27421592,  0.24393222, -0.13974937, -0.09739598, -0.07313758,
  0.18198089]
var_training_data =[0.75261122, 0.90482986, 0.91105327, 0.75504036, 1.07026701, 0.76708319,
 0.35172769, 0.83004988, 0.76964675, 0.57386915, 0.45912309, 0.2955709,
 1.61493449]

### Mean and Variance from artifical Dataset

In [ ]:
mean_training_data = [-0.00393712, -0.01294209]
var_training_data = [49.18936568,  0.34270256]

### Mean and Variance form cpps Dataset

In [ ]:
mean_training_data = [0.05330162, 0.03075699, -0.05636937, 0.0274802, 0.06536314, -0.04620979,-0.0745559, 
                      -0.08149049, -0.05318843, 0.11105582]
var_training_data = [0.02905961, 0.04473883, 0.05254194, 0.05198144, 0.07337494, 0.0666981, 0.07593811, 
                     0.0393896, 0.08028017, 0.0594492]

## Create DataLoader

In [4]:
data_preperator = DataPreperatorPrediction(path=param['data']['path'], 
                                           ignored_features = param["data"]["droped_feature"],
                                           mean_training_data=mean_training_data, 
                                           var_training_data=var_training_data, 
                                           first_order_difference=False 
                                          )                                  
preprocessed_data = data_preperator.prepare_data()
print(preprocessed_data.shape)

dataset = DataSet(preprocessed_data, 
                  timesteps=param["model"]["sequence_size"])
data_loader = DataLoader(dataset, 
                         batch_size=param['model']['batch_size'], 
                         num_workers=0, 
                         shuffle=False, 
                         drop_last=True)

(12502, 14)


## Define Model and load Parameters of trained model

In [5]:
model = LstmMultiTaskLearning(batch_size=param['model']['batch_size'], 
                              input_dim=param['model']['input_size'], 
                              n_hidden_lstm=param['model']['n_hidden_lstm'], 
                              n_layers=param['model']['lstm_layer'],
                              dropout_rate_lstm= param['model']['dropout_rate_lstm'],
                              dropout_rate_fc= param['model']['dropout_rate_fc'],
                              n_hidden_fc_prediction=param['model']['n_hidden_fc_pred'], 
                              n_hidden_fc_ls_analysis=param['model']['n_hidden_fc_ls']      
                              )

checkpoint = torch.load(param["model"]["path"])
model.load_state_dict(checkpoint['model_state_dict'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

## Define Loss Function

In [6]:
criterion = LossMseInference(param["model"]["input_size"], param["model"]["batch_size"])

## Initialize Predictor

In [10]:
predictor = PredictorMultiTaskLearning(model=model,
                                       criterion=criterion,
                                       path_data=param["data"]["path"],
                                       columns_to_ignore=param["data"]["droped_feature"],
                                       threshold_anomaly=param["anomaly_detection"]["threshold_anomaly"]
                                      )

## Predict

In [11]:
print("Start predicting.")
# Write header
with open(param["results"]["path"], "a+") as file:
            [file.write(column+";") for column in predictor.create_column_names_result()]
            file.write("\n")

for batch_number, (input_data, target_data) in enumerate(data_loader):
    # Predict sensor values in mini-batches
    batch_results = predictor.predict(input_data, target_data)
    
    # Write results to csv file
    with open(param["results"]["path"], "a") as file:
        for batch in batch_results:
            # Each result component of a singe prediction (ID, target, prediction, loss, latent space ...) is stored in lists
            # thus we have to unpack the list and seperate values with ;
            for value in batch:
                file.write(str(value)+";")
            file.write("\n")

    # Print status 
    if (batch_number*param['model']['batch_size'])%5000 == 0:
        print("Current status: " + str(param['model']['batch_size']*batch_number) + " samples are predicted.")

print("End of prediction.")

Start predicting.
Current status: 0 samples are predicted.
Current status: 5000 samples are predicted.
Current status: 10000 samples are predicted.
End of prediction.


## Tag anomalous samples

In [12]:
results_prediction = pd.read_csv(param["results"]["path"], sep=";")
# Values of column "loss" are exponentially smoothed and stored in a new column "smoothed loss"
# New column "anomaly" is created and sample is taged with 1 if anomalous behaviour (if smoothed loss is over threshold)
results = predictor.detect_anomaly(results_prediction, param["anomaly_detection"]["smooth_rate"])

## Combine prediction data with data which was not consider for inference

In [13]:
original_sensor_data = pd.read_csv(param["data"]["path"])
data_of_droped_feature = original_sensor_data.loc[:, param["data"]["droped_feature"]+["ID"]]
complete_data = results.merge(right=data_of_droped_feature, how="inner", on="ID")

## Save data to csv file

In [14]:
complete_data.to_csv(param["results"]["path"], sep=";", index=False)